In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults
from typing import TypedDict, Annotated,List
from langgraph.graph import add_messages, StateGraph, END
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod
from pydantic import BaseModel

In [2]:
load_dotenv()

True

In [34]:
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]

In [35]:
search_tool=TavilySearchResults(max_results=2)

In [36]:
tools=[search_tool]

In [37]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [38]:
llm_with_tools=llmGemini.bind_tools(tools=tools)

In [39]:
def model(state: AgentState):
    state["messages"]=[llm_with_tools.invoke(input=state["messages"])]
    return state
    

In [40]:
def tools_router(state:AgentState):
    last_message=state["messages"][-1]

    if hasattr(last_message,"tool_calls") and len(last_message.tool_calls)>0:
        return "tool_node"
    else:
        return "end"

In [41]:
tool_node=ToolNode(tools=tools,messages_key="messages")

In [42]:
graph=StateGraph(state_schema=AgentState)

In [43]:
graph.add_node(node="model",action=model)

In [44]:
graph.add_node(node="tool_node",action=tool_node)

In [45]:
graph.set_entry_point(key="model")

In [46]:
graph.add_conditional_edges(source="model",path=tools_router,path_map={"tool_node":"tool_node","end":END})

In [47]:
graph.add_edge(start_key="tool_node",end_key="model")

In [48]:
app=graph.compile()

In [49]:
print(app.get_graph().draw_ascii())

          +-----------+           
          | __start__ |           
          +-----------+           
                 *                
                 *                
                 *                
            +-------+             
            | model |             
            +-------+.            
           **         ..          
         **             ..        
        *                 .       
+-----------+         +---------+ 
| tool_node |         | __end__ | 
+-----------+         +---------+ 


In [50]:
display(Image(data=app.get_graph(xray=True).draw_mermaid_png()))

KeyboardInterrupt: 

In [51]:
input={
    "messages":["What's the current weather in Bangalore?"]
}

In [52]:
app.invoke(input=input)

{'messages': [HumanMessage(content="What's the current weather in Bangalore?", additional_kwargs={}, response_metadata={}, id='2fd824f9-8fe3-49d5-8ed3-2a9e502847cf'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Bangalore"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-9c9adf99-35d5-41da-8412-b05ba606edda-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Bangalore'}, 'id': '4ffb8f90-7b5a-4793-8a8d-c9c7a05819ff', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"title": "Bengaluru Weather Forecast 18 May 2025 - Times of India", "url": "https://timesofindia.indiatimes.com/weather/bengaluru-weather-forecast-today/5600

In [53]:
events=app.stream(input=input, stream_mode="values")

In [54]:
for event in events:
    print(event,"\n")

{'messages': [HumanMessage(content="What's the current weather in Bangalore?", additional_kwargs={}, response_metadata={}, id='8c64d29b-394b-4ea1-bfd9-01f8bcdb3ecd')]} 

{'messages': [HumanMessage(content="What's the current weather in Bangalore?", additional_kwargs={}, response_metadata={}, id='8c64d29b-394b-4ea1-bfd9-01f8bcdb3ecd'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Bangalore"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-57b6cf67-64c6-45b6-9981-dafc085a0917-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Bangalore'}, 'id': 'cdf2ee13-9626-4ed4-8bcb-6ea3516e9fe6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})]} 

{'messages'

In [55]:
events=app.stream(input=input, stream_mode="updates")

In [56]:
for event in events:
    print(event,"\n")

{'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Bangalore"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-c4254f7e-0db7-4df0-b4b6-3e200f9a29e6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Bangalore'}, 'id': '43960402-3c6a-44f9-9bad-3d941bc0cd21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})]}} 

{'tool_node': {'messages': [ToolMessage(content='[{"title": "Bengaluru Weather Forecast 18 May 2025 - Times of India", "url": "https://timesofindia.indiatimes.com/weather/bengaluru-weather-forecast-today/560001", "content": "Today\'s Weather in Bengaluru: In Bengaluru today, the weather is expected to be Partly Cloudy wit

In [57]:
input={
    "messages":["Hi how are you?"]
}

In [58]:
events=app.astream_events(input=input, version="v2")

In [59]:
async for event in events:
    print(event,"\n")

{'event': 'on_chain_start', 'data': {'input': {'messages': ['Hi how are you?']}}, 'name': 'LangGraph', 'tags': [], 'run_id': 'e1f5f870-6a32-4027-b932-99e13a4ea104', 'metadata': {}, 'parent_ids': []} 

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hi how are you?', additional_kwargs={}, response_metadata={}, id='5597075d-0aba-4b48-aeb1-56ba2a88787c')]}}, 'name': 'model', 'tags': ['graph:step:1'], 'run_id': '39837551-9ae9-4adc-a649-2ba255175db4', 'metadata': {'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:5a6bc4f7-ad45-461a-1ee6-7d92718d4a23'}, 'parent_ids': ['e1f5f870-6a32-4027-b932-99e13a4ea104']} 

{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='Hi how are you?', additional_kwargs={}, response_metadata={}, id='5597075d-0aba-4b48-aeb1-56ba2a88787c')]]}}, 'name': 'ChatGoogleGenerativeAI', 

In [60]:
events=app.astream_events(input=input, version="v2")

In [61]:
async for event in events:
    if event["event"]=="on_chat_model_stream":
        print(event['data']['chunk'].content,end="",flush=True)

I am doing well, thank you for asking! How can I help you today?